# Scrape Bible Summary Tweets
Gregory Antell

July 2017

# Import requirements

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import sqlite3
import sqlalchemy
import matplotlib.pyplot as plt
%matplotlib inline

# Functions

In [48]:
def get_chapter_tweets(book_url):
    """
    returns all tweets for a book
    """
    page = requests.get(book_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    chapter_list, tweet_list = [], []
    ch_num = 1
    for tag in soup.find_all("p", class_="tweet_content"):
        raw_text = tag.text
        label, split, new_text = raw_text.partition(': ')
        chapter_list.append(ch_num)
        ch_num += 1
        tweet_list.append(str(new_text))
    return chapter_list, tweet_list

# Get book URLs

In [33]:
main_url = "http://www.biblesummary.info"
page = requests.get(main_url)
soup = BeautifulSoup(page.content, 'html.parser')

# create book dictionary to store urls
book_dict = {'book':[],'url':[]}
for ultag in soup.find_all("ul", class_="bible"):
    for litag in ultag.find_all("li"):
        for atag in litag.find_all("a"):
            book_dict['book'].append(atag.text)
            book_dict['url'].append(main_url + atag['href'])

# Get tweets for each chapter

In [60]:
# create chapter dictionary to store summaries
chapter_dict = {'book':[], 'chapter':[], 'tweet':[]}

for book_info in zip(book_dict['book'], book_dict['url']):
    book_name = str(book_info[0])
    book_url = book_info[1]    
    chapter_list, tweet_list = get_chapter_tweets(book_url)
    chapter_dict['book'].extend([book_name]*len(chapter_list))
    chapter_dict['chapter'].extend(chapter_list)
    chapter_dict['tweet'].extend(tweet_list)

chapter_tweet_df = pd.DataFrame(chapter_dict)

In [61]:
print chapter_tweet_df.shape
chapter_tweet_df.head()

(1189, 3)


,book,chapter,tweet
0,Genesis,1,"God created the heavens, the earth and everyth..."
1,Genesis,2,God formed a man and gave him the garden in Ed...
2,Genesis,3,The serpent deceived the woman. She and Adam a...
3,Genesis,4,Eve's sons made offerings to the LORD. Only Ab...
4,Genesis,5,"Adam's line was: Seth, Enosh, Kenan, Mahalalel..."


# Move to SQL